# Parse Ministry Grid for GENESIS Kids usage

Little util to quickly parse out what we use

In [ ]:
!pip install striprtf
!pip install python-docx
!pip install beautifulsoup4
!pip install pillow
import re


In [2]:
import re
from striprtf.striprtf import rtf_to_text

def get_section(start, end, text):
  return re.search(rf'({start}.*?){end}', text, re.DOTALL).group(1).strip()

def parse_document(file_path):
    with open(file_path, 'r') as f:
      rtf_text = f.read()

    text = rtf_to_text(rtf_text)

    matches = re.findall(r'(UNIT \d+\s\s\|\s\sSESSION \d+.*?)(?=Journal and prayer)', text, re.DOTALL)

    pages = []
    for i, match in enumerate(matches):
      header = get_section('UNIT \d+\s\s\|\s\sSESSION \d+.*?', 'PREPARE', match)
      leader = get_section('Leader Bible Study', 'GATHER Supplies', match)
      large_activities = get_section('SESSION STARTER', 'LEARN', match)
      intro = get_section('Introduce the session.*?SAY', 'Big picture question', match)
      big_picture = get_section('Big picture question.*?SAY', 'Review the timeline', match)
      review_timeline = get_section('Review the timeline.*?SAY', 'LEARN', match)
      christ_connection = get_section('Christ connection.*?SAY', '\(Tip:', match)
      small_group = get_section('GROUP DISCUSSION', 'LIVE', match)
      small_group = re.sub(r'\(Option: (.*?)\)', r'\1\n', small_group)
      small_activities = get_section('Activity choice', '$', match)

      pages.append({
          'match': match,
          'header': header,
          'leader': leader,
          'large_activities': large_activities,
          'intro': intro,
          'big_picture': big_picture,
          'review_timeline': review_timeline,
          'christ_connection': christ_connection,
          'small_group': small_group,
          'small_activities': small_activities
      })

      header1 = get_section('(UNIT \d+\s\s\|\s\sSESSION \d+.*?)', 'PREPARE', match)
      if header1.startswith('UNIT 17  |  SESSION 3'):
          # print(f'####\n{match}\n####')
          print(f'# Header:\n{header}\n')
          print(f'# Leader:\n{leader}\n')
          print(f'# Large Activities:\n{large_activities}\n')
          print(f'# Intro:\n{intro}\n')
          print(f'# Big Picture:\n{big_picture}\n')
          print(f'# Review Timeline:\n{review_timeline}\n')
          print(f'# Christ Connection:\n{christ_connection}\n')
          print(f'# Small Group:\n{small_group}\n')
          print(f'# Small Activities:\n{small_activities}\n')

    return pages

docs = parse_document('./02_tgp4_v6_ok_leader_guide.rtf')

# Header:
UNIT 17  |  SESSION 3
The People Rebuilt the Temple
Ezra 2–6
Key Passage:
Jeremiah 17:7
Big Picture Question:
Can God be trusted? Yes, God cannot lie or ever be wrong, so we can trust whatever He has said.
Christ Connection: 
God’s people rebuilt the temple so they would have a place to worship God. Years later, God sent His Son, Jesus, to be with His people. Now, God does not dwell with His people through the temple. Instead, God dwells directly with His people through His Spirit. Jesus provided something better than the temple; He gives us Himself.

# Leader:
Leader Bible Study
Imagine this for a moment: It’s late Saturday night, and you’ve been out of the house all day. You’ve gone from place to place, striving to complete more than seemed possible from your calendar to-do list. You’ve just returned home and sat down for dinner. Suddenly, you remember that tomorrow is Sunday. You quickly recall that you are scheduled to serve with preschoolers tomorrow morning, your small 

In [27]:
import os
import httpx
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

prompt_templates = {
  'html_verse': 'You are a GPT. Below is an HTML object which contains a Bible verse. You should only return the Bible verse and the reference, follow the format:\n"verse text" - reference.',
  'key_passage_worship': '''You are a GPT, using reformed theology. You will be provided a Bible verse, you need to review the verse and provide one short insight into why we should worship \
God based on that verse. Focus on God's attributes. This insight will be read by a Sunday School teacher to a class of 1st through 3rd graders.''',
  'activities': '''You are a GPT. You will be provided a scripture reference and various activities for a Sunday School class. You need to review the options, decide if one of these will suffice. \
Refer to ABOUT THE CLASS and based your activity on the guidelines in IDEAL ACTIVITY section. If not, please create a new activity.

ABOUT THE CLASS:
- We are a Sunday School class in a reformed church.
- We are in a large room that fits 30 kids at most, comfortably.
- On average our classes have 10-20 kids.
- The kids are in 2nd and 3rd grade.
- We have typical classroom supplies and have budget to purchase small items.

IDEAL ACTIVITY:
- The activity should be engaging, could be high paced, or slow and methodical.
- The activity can be directly related to the scripture reference, but that is not necessary.
- The activity can have some prepation time, keep it under 10 minutes.
- The activity should be able to be completed in 10-15 minutes.

Once you've decided on an activity, please provide it back to the user in the following format with no markdown formatting, introduction or explanation:

MATERIALS:
- List of materials needed for the activity.
- Include preparation steps if necessary.

OVERVIEW:
- Describe the game briefly for the teacher to understand how to play the game.

INSTRUCTIONS:
- Define the instructions for the teacher to read to the kids.
- Anytime you want the teach to speak to the kids, preface that sentence with SAY
'''
}

def call_gpt_instruct(message, prompt_template):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}",
    }

    request_body = {
        "prompt": f"{prompt_templates[prompt_template]}\n\nHTML Object:\n{message}",
        "model": "gpt-3.5-turbo-instruct", # "gpt-4-turbo-preview",
        "temperature": 0.8,
        "top_p": 0.95,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "max_tokens": 800,
        "stream": False,
    }
    openai_url = "https://api.openai.com/v1/completions"
    response = httpx.post(openai_url, json=request_body, headers=headers)
    if response.status_code != 200:
        print(response.text)
    return response.json()['choices'][0]['text'].strip()


def call_gpt(message, prompt_template):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}",
    }

    request_body = {
        "messages": [
            {
                "role": "system",
                "content": prompt_templates[prompt_template]
            },
            {
                "role": "user",
                "content": message
            }
        ],
        "model": "gpt-4-turbo-preview",
        "temperature": 0.8,
        "top_p": 0.95,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "max_tokens": 1000,
        "stream": False,
    }
    openai_url = "https://api.openai.com/v1/chat/completions"
    response = httpx.post(openai_url, json=request_body, headers=headers, timeout=30)
    if response.status_code != 200:
        print(response.text)
    return response.json()['choices'][0]['message']['content'].strip()


In [8]:
print(call_gpt('"He will take delight in you with gladness. With his love, he will calm all your fears." - Zephaniah 3:17, NLT', 'key_passage_worship'))
print(call_gpt('"But blessed are those who trust in the Lord" - Jeremiah 17:7, NLT', 'key_passage_worship'))
print(call_gpt('"For the kind of sorrow God wants us to experience leads us away from sin and results in salvation. There’s no regret for that kind of sorrow. But worldly sorrow, which lacks repentance, results in spiritual death." - 2 Corinthians 7:10, NLT)', 'key_passage_worship'))


God loves us so much that He is happy to be with us and takes away all our worries with His love. This shows us that God is loving and caring, and that's why we should worship Him.
"God is like the best friend who is always there for us. When we trust in God, He makes us happy and fills our hearts with joy, just like when we feel safe and loved by our best friends. That's why we worship God, because He is trustworthy and loves us so much."
God loves us so much that He helps us feel sorry for doing wrong things, not to make us sad, but to help us make better choices that lead us to be with Him forever. We should worship God because He is kind and leads us to what is truly good for us.


In [23]:
import httpx


verse_cache = []
def get_verse(doc):
    global verse_cache
    verse = doc['header'].splitlines()[4]
    if verse in [v['verse'] for v in verse_cache]:
        doc['verse_text'] = [v['verse_text'] for v in verse_cache if v['verse'] == verse][0]
    else:
        # print(verse)
        url = "https://api.nlt.to/api/passages"
        params = {
          "ref": verse,
          "key": "lozzi"
        }

        response = httpx.get(url, params=params)
        # print(response.text)
        verse_text = call_gpt_instruct(response.text, 'html_verse')
        print(verse_text)
        doc['verse_text'] = verse_text
        verse_cache.append({
            'verse': verse,
            'verse_text': verse_text
        })

for doc in docs:
    get_verse(doc)

"He is a mighty savior. He will take delight in you with gladness. With his love, he will calm all your fears." - Zephaniah 3:17, NLT
"But blessed are those who trust in the Lord" - Jeremiah 17:7, NLT
"For the kind of sorrow God wants us to experience leads us away from sin and results in salvation. There’s no regret for that kind of sorrow. But worldly sorrow, which lacks repentance, results in spiritual death." - 2 Corinthians 7:10, NLT


In [10]:
print("\n".join([doc['verse_text'] for doc in docs]))

"He will take delight in you with gladness. With his love, he will calm all your fears." - Zephaniah 3:17, NLT
"He will take delight in you with gladness. With his love, he will calm all your fears." - Zephaniah 3:17, NLT
"He will take delight in you with gladness. With his love, he will calm all your fears." - Zephaniah 3:17, NLT
"He will take delight in you with gladness. With his love, he will calm all your fears." - Zephaniah 3:17, NLT
"But blessed are those who trust in the Lord" - Jeremiah 17:7, NLT
"But blessed are those who trust in the Lord" - Jeremiah 17:7, NLT
"But blessed are those who trust in the Lord" - Jeremiah 17:7, NLT
"But blessed are those who trust in the Lord" - Jeremiah 17:7, NLT
"For the kind of sorrow God wants us to experience leads us away from sin and results in salvation. There's no regret for that kind of sorrow. But worldly sorrow, which lacks repentance, results in spiritual death." - 2 Corinthians 7:10, NLT
"For the kind of sorrow God wants us to experi

In [10]:
from docx import Document

def get_unit_session(unit_session):
  pattern = r"UNIT (\d+)\s*\|\s*SESSION (\d+)"
  match = re.search(pattern, unit_session)

  if match:
      unit = match.group(1)
      session = match.group(2)
      output = f"u{unit}s{session}"

      return output

def get_unit_session_nums(unit_session):
  pattern = r"UNIT (\d+)\s*\|\s*SESSION (\d+)"
  match = re.search(pattern, unit_session)

  if match:
      unit = match.group(1)
      session = match.group(2)

      return int(unit), int(session)

def create_doc(doc):
    header_lines = doc['header'].splitlines()
    unit_session = header_lines[0].upper()
    unit, session = get_unit_session_nums(unit_session)
    if unit < 17:
       return

    doc_path = 'template.docx'

    template = Document(doc_path)

    leader_lines = doc['leader'].splitlines()
    big_picture_lines = doc['big_picture'].splitlines()
    intro_lines = doc['intro'].splitlines()
    review_timeline_lines = doc['review_timeline'].splitlines()
    small_group_lines = doc['small_group'].splitlines()
    key_passage_worship = call_gpt(doc['verse_text'], 'key_passage_worship')

    activity_msg = f"""Scripture reference: {header_lines[2]}
Activities:
{doc['large_activities']}
{doc['small_activities']}
"""
    gpt_activities = call_gpt(activity_msg, 'activities')

    replacements = {
        '{{HEADER}}': unit_session,
        '{{TITLE}}': header_lines[1].upper(),
        '{{REFERENCE}}': header_lines[2].upper(),
        '{{BOOK_REFERENCE}}': header_lines[2].split(' ')[0].upper(),
        '{{KEY_PASSAGE}}': doc['verse_text'],
        '{{KEY_PASSAGE_WORSHIP}}': key_passage_worship,
        '{{BIG_PICTURE_QUESTION}}': header_lines[6],
        '{{CHRIST_CONNECTION}}': header_lines[8],

        '{{LEADER_BIBLE_STUDY}}': "\n\n".join(leader_lines[1:len(leader_lines)-2]),
        '{{LEADER_TO_KNOW}}': "\n".join(leader_lines[len(leader_lines)-2:]),

        '{{BIG_QUESTION_TALK}}': "\n".join(big_picture_lines[1:]),

        '{{INTRO}}': "\n\n".join(intro_lines[3:]),

        '{{REVIEW}}': "\n\n".join(review_timeline_lines[2:]),

        '{{GPT_ACTIVITIES}}': gpt_activities,
        '{{LARGE_ACTIVITIES}}': doc['large_activities'],
        '{{SMALL_ACTIVITIES}}': doc['small_activities'],

        '{{SMALL_GROUP}}': "\n\n".join(small_group_lines[1:]),
    }

    # display(replacements)

    for shape in template.element.xpath('.//w:txbxContent//w:p'):
      for paragraph in shape.iterchildren():
        for run in paragraph.iterchildren():
          if run.text is not None:
            print(run.text)
            for placeholder, replacement in replacements.items():
              if placeholder in run.text:
                print(f'found {placeholder}')
                run.text = run.text.replace(placeholder, replacement)

    for paragraph in template.paragraphs:
        for placeholder, replacement in replacements.items():
          if placeholder in paragraph.text:
            print(f'found {placeholder}')
            paragraph.text = paragraph.text.replace(placeholder, replacement)


    template.save(f'leader guide {get_unit_session(unit_session)}.docx')

for doc in docs:
   create_doc(doc)


In [15]:
from PIL import Image, ImageDraw, ImageFont
import textwrap
import os

files = os.listdir('.')
for doc in docs:
    header_lines = doc['header'].splitlines()
    unit_session = header_lines[0].upper()
    unit, session = get_unit_session_nums(unit_session)

    if unit > 17:
        jpg_file = [file for file in files if file.endswith('.jpg') and f'u{unit}_s{session}' in file][0]

        img = Image.open(jpg_file)
        img.save(f'u{unit} s{session} title blank.jpg')

        draw = ImageDraw.Draw(img)

        font_size = 200
        font = ImageFont.truetype('Roboto-Bold.ttf', font_size)

        # Specify text
        text = header_lines[1]

        # Wrap the text
        lines = textwrap.wrap(text, width=30)

        y_text = img.height - (len(lines) * font_size) - 50 #margin
        for line in lines:
            # Calculate the width and height of the text
            top, left, text_width, text_height = font.getbbox(line)

            x = (img.width - text_width) / 2
            y = y_text #position at the bottom

            shadowcolor = "black"
            offset = 8
            draw.text((x+offset, y_text+offset), line, font=font, fill=shadowcolor)

            # Draw text on image
            draw.text((x, y), line, font=font, fill="white")
            y_text += font_size


        # Save the image
        img.save(f'u{unit} s{session} title.jpg')

In [ ]:
import os

# List all files in the current directory
files = os.listdir('.')

# Filter out the files that end with .jpg and contain u18_s1
jpg_files = [file for file in files if file.endswith('.jpg') and 'u18_s1' in file]

print(jpg_files)

In [7]:
import json
display(json.dumps(docs))

'[{"match": "UNIT 16  |  SESSION 1\\nJeremiah Encouraged God\\u2019s People\\nJeremiah 29\\u201330\\nKey Passage:\\nZephaniah 3:17\\nBig Picture Question:\\nWhere is God? God is in all places at all times and is always with His people.\\nChrist Connection: \\nGod did not abandon His people. The prophet Jeremiah encouraged God\\u2019s people. He said God was going to save His people from captivity and raise up a new King\\u2014a forever King\\u2014from David\\u2019s family.\\n\\n\\n\\nPREPARE\\nPrior TO TEACHING THE SESSION\\n\\nLeader Bible Study\\nGod\\u2019s Word is living, active, sharp, powerful, and profitable. (Heb. 4:12; 2 Tim. 3:16-17) The Bible indicates that by the power of the Holy Spirit, God\\u2019s words teach, rebuke, correct, and train us so that we are complete and equipped for the work that God has created us for. The Bible is full of stories, illustrations, commands, warnings, and heart-provoking questions that provide for us everything we need.\\nYet, throughout our

'Jeremiah Encouraged God’s People'